# Cleaning Data

**1)** For our data cleaning exercises, we will return one last time to our ACS data [here](https://github.com/nickeubank/MIDS_Data/tree/master/US_AmericanCommunitySurvey). Download and import the 10percent ACS sample.

In [1]:
import pandas as pd
import numpy as np
# Download the data
acs = pd.read_stata("https://github.com/nickeubank/MIDS_Data/raw/master/US_AmericanCommunitySurvey/US_ACS_2017_10pct_sample.dta")

**2)** For our exercises today, we'll focus on `age`, `gender`, `educ`, and `inctot`. Subset your data to those variables, and quickly look at a sample of 10 rows. 

In [2]:
acs = acs[['age', 'sex', 'educ', 'inctot']]

In [3]:
acs.sample(10)

age     sex                 educ  inctot
99185   26    male             grade 12       0
317882  15    male              grade 9       0
286926  57  female             grade 12   48800
97900   25  female             grade 12    8000
244210  69  female             grade 12  132700
81911   28  female   4 years of college   57000
16771   68  female             grade 12   23750
106052  35    male             grade 12  160000
225796  56    male             grade 12   40000
83623   60    male  5+ years of college  140000

**3)** First, replace all the values of `inctot` that are 9999999 with `np.nan`. 

In [4]:
acs['inctot'] = acs['inctot'].replace(9999999, np.nan)

**4)** So we know how data is being stored, check the `dtypes` of all the variables we are working with. What is the `dtype` of `age`?

In [5]:
acs.dtypes

age       category
sex       category
educ      category
inctot     float64
dtype: object

**5)** We want to be able to calculate things using age, so we need it to be a numeric type. Check all the values of `age` to figure out why it's categorical and not numeric. You should find two problematic categories. 

In [6]:
acs['age'].value_counts()

60                           4950
54                           4821
59                           4776
56                           4776
58                           4734
57                           4720
55                           4693
61                           4644
62                           4614
53                           4600
18                           4496
63                           4488
52                           4418
65                           4362
19                           4342
64                           4287
50                           4272
47                           4256
16                           4106
66                           4106
46                           4064
67                           4055
17                           4021
51                           4021
10                           3997
20                           3992
9                            3977
48                           3956
70                           3953
68            

**6)** In order to convert `age` into a numeric variable, we need to replace those problematic entries with values that `pandas` can later convert into numbers. Pick appropriate substitutions for the existing values and replace the current values. **Hint 1:** Categorical variables act like strings, so you might want to use string methods! **Hint 2:** Remember that characters like parentheses, pluses, asterices, etc. are special in Python strings, and you have to escape them if you want them to be interpreted literally!

In [7]:
acs['age'] = acs['age'].str.replace("less than 1 year old", '0')
acs['age'] = acs['age'].str.replace("90 \(90\+ in 1980 and 1990\)", '90')

In [8]:
acs['age'].value_counts()

60    4950
54    4821
56    4776
59    4776
58    4734
57    4720
55    4693
61    4644
62    4614
53    4600
18    4496
63    4488
52    4418
65    4362
19    4342
64    4287
50    4272
47    4256
16    4106
66    4106
46    4064
67    4055
51    4021
17    4021
10    3997
20    3992
9     3977
48    3956
70    3953
68    3951
      ... 
1     3340
4     3318
3     3220
0     3150
71    2917
72    2901
74    2819
73    2781
75    2532
76    2170
77    2089
78    1985
79    1758
80    1721
81    1524
82    1464
83    1335
84    1157
85    1117
86    1041
94    1035
87     908
88     859
89     628
90     480
93     476
95     471
92     355
91     227
96      10
Name: age, Length: 97, dtype: int64

**7)** Now convert age from a categorical to numeric. 

In [9]:
acs['age'] = acs['age'].astype('int')

**8)** Let's now filter out anyone in our data whose age is greater than 18. Note that before made `age` a numeric variable, we couldn't do this!

In [10]:
acs = acs[acs['age'] > 18]

**9)** Create an indicator variable for whether each person has at least a college degree called `college_degree`. 

In [18]:
acs['educ'].value_counts()

grade 12                     89743
4 years of college           47212
1 year of college            38384
5+ years of college          29801
2 years of college           20731
grade 5, 6, 7, or 8           5942
grade 11                      4763
grade 10                      3942
n/a or no schooling           3627
grade 9                       3105
nursery school to grade 4     1288
Name: educ, dtype: int64

In [19]:
acs['college_degree'] = (acs['educ'] == '4 years of college') | (acs['educ'] == '5\+ years of college')
acs['college_degree'].value_counts()

False    201326
True      47212
Name: college_degree, dtype: int64

**10)** Let's examine how the educational gender gap. Use `pd.crosstab` to create a cross-tabulation of `sex` and `college_degree`. `pd.crosstab` will give you the number of people who have each combination of `sex` and `college_degree` (so in this case, it will give us a 2x2 table with Male and Female as rows, and `college_degree` True and False as columns, or vice versa. 

In [22]:
pd.crosstab(acs['college_degree'], acs['sex'])

sex              male  female
college_degree               
False           97723  103603
True            22039   25173

**11)** Counts are kind of hard to interpret. `pd.crosstab` can also normalize values to give percentages. Look at the `pd.crosstab` help file to figure out how to normalize the values in the table. Normalize them so that you get the share of men with and without college degree, and the share of women with and without college degrees. 

In [23]:
pd.crosstab(acs['college_degree'], acs['sex'], normalize='columns')

sex                 male    female
college_degree                    
False           0.815977  0.804521
True            0.184023  0.195479

**12)** Now, let's recreate that table for people over 40 and people under 40. Has the difference between men and women in terms of getting a college degree impoved, stayed the same, or worsened?

In [24]:
older = acs[acs['age'] > 40]
pd.crosstab(older['college_degree'], older['sex'], normalize='columns')

sex                 male    female
college_degree                    
False           0.819964  0.824193
True            0.180036  0.175807

In [25]:
older = acs[acs['age'] < 40]
pd.crosstab(older['college_degree'], older['sex'], normalize='columns')

sex                 male    female
college_degree                    
False           0.808985  0.763971
True            0.191015  0.236029